### MLP（多层感知机模型）

在bigram设计中，只考虑了前一个字符作为上下文（context），导致模型预测的效果不好。解决方法是要考虑更多的上下文来预测序列中的下一个字符。但是我们发现情况会变得很复杂，这张表格的大小会迅速增长，实际上它会随着上下文长度的增加呈指数级增长。

MLP用于预测序列（sequence）中的下一个字符

In [6]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [7]:
# read in all the words
words = open("names.txt", 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [8]:
len(words)

32033

In [13]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [90]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
#for w in words[:5]:
for w in words:
    
    #print(w)
    context = [0] * block_size # initialize context block 0->'.'
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append
        
X = torch.tensor(X)
Y = torch.tensor(Y)

X是输入示例（context block），Y是标签（label），即紧接着上下文窗口的下一个字符

In [40]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [91]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

##### 构建特征矩阵C，即词嵌入查找表（embedding lookup table C）

table C有27个可能的字符，我们将其嵌入到一个较低维度的空间中（这里是二维）

In [28]:
C = torch.randn((27, 2))

In [35]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

这里的`C[X]`用到了**张量的索引操作**：使用张量`X`作为索引去访问张量`C`，其行为取决于索引张量`X`的形状和数据类型。`C[X]`中，`X`充当**行索引**（32x3=96个范围在`[0,26]`的整数索引值，故可用于`C`的行索引），保留`C`的列维度，最终可得出`C[X]`形状为`[32, 3, 2]`

In [60]:
# 隐藏层：该层的输入数量为3x2，因为有三个block，每个block有两个维度的嵌入

W1 = torch.randn((6, 100)) # hyperparameter是自定义的
b1 = torch.randn(100)

为了避免硬编码数值32，这里使用`emb.shape[0]`或者直接`-1`让pytorch自己推理出32。以下都是等价的写法：

```python
h = torch.tanh(emb.view(32, 6) @ W1 + b1)
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
```

======================================

In [37]:
emb @ W1 + b1

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [43]:
# 举例，将3个词嵌入通过“连接”压缩成了32x6的形状
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape 

torch.Size([32, 6])

In [48]:
# 将emb张量移除1维度，得到一组序列后通过“连接”压缩成了32x6
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([32, 6])

In [55]:
emb.shape

torch.Size([32, 3, 2])

In [ ]:
emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

======================================

为了避免硬编码数值32，这里使用`emb.shape[0]`或者直接`-1`让pytorch自己推理出32。以下都是等价的写法：

```python
h = torch.tanh(emb.view(32, 6) @ W1 + b1)
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
```
另外还需注意这里的`+ b1`用到了**广播机制**，因为矩阵`emb.view(-1, 6) @ W1`的形状是`[32, 100]`，`b1`的形状是`[100]`，两者不能直接相乘。在广播下，`b1`的形状拓展为`[1, 100]`，此时相乘，每个`b1`的偏置向量都会被加到这个矩阵的所有行上

In [64]:
# hidden layer of activations

h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [65]:
h

tensor([[-0.6893, -0.3969, -0.9974,  ..., -0.8104, -0.5748, -0.9779],
        [-0.3288,  0.6972, -0.9906,  ..., -0.8977, -0.4107, -0.9917],
        [-0.8146,  0.9576, -0.9853,  ..., -0.9982, -0.9414, -0.9748],
        ...,
        [ 0.7185, -0.5837, -0.8713,  ..., -0.8332, -0.4186,  0.7160],
        [ 0.3113, -0.9968, -0.9938,  ...,  0.9818,  0.9305, -0.9808],
        [ 0.1610,  0.9884, -0.9982,  ..., -0.9568, -0.2409, -0.3670]])

In [66]:
h.shape

torch.Size([32, 100])

In [67]:
# output layer: 

W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [68]:
logits = h @ W2 + b2

In [69]:
counts = logits.exp()

In [70]:
prob = counts / counts.sum(1, keepdim=True)

In [71]:
prob.shape

torch.Size([32, 27])

In [73]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(20.6028)

In [ ]:
# --------------------- now made repectable :) ----------------------

In [128]:
X.shape, Y.shape # dataset

(torch.Size([228146, 3]), torch.Size([228146]))

In [129]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
# hidden layer 's weights and bias
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
# output layer 's weights and bias
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [130]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [131]:
for p in parameters:
    p.requires_grad = True

In [139]:
for _ in range(1000):
    
    # minibatch construct
    ix = torch.randint(0,X.shape[0],(32,))
    
    # forward pass
    emb = C[X[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Y[ix])
    #print(loss.item())
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.1 * p.grad

print(loss.item())

2.456792116165161


一种确定合理学习率的方法